In [1]:
from myimports import *

In [2]:
# Demo 1: Set and run an experiment

# Set the experiment
setting1 = SynchronousByzantine(n=5, input=1,
                               protocol=NaiveVoting,
                               measure=[ByzValidity, ByzConsistency])
exp1 = Experiment(setting1)

# Go
exp1.run_and_report()

Experiment #1
Experiment Setting:
Synchronous Byzantine
Controller : Synchronous Byzantine Controller
Environment : Synchronous Byzantine Environment
PKI : IdealPKI
Protocol : Naive Voting Protocol

Parameters:
n : 5
input : 1

Experiment Result:
Node 0 : 1
Node 1 : 1
Node 2 : 1
Node 3 : 1
Node 4 : 1

Measurements:
Validity : True
Consistency : True

Message History :
Round 1 : 
From 0 to 0 content 1|0 
From 0 to 1 content 1|0 
From 0 to 2 content 1|0 
From 0 to 3 content 1|0 
From 0 to 4 content 1|0 
Round 2 : 
From 0 to 0 content 1|0|0 
From 0 to 1 content 1|0|0 
From 0 to 2 content 1|0|0 
From 0 to 3 content 1|0|0 
From 0 to 4 content 1|0|0 
From 1 to 0 content 1|0|1 
From 1 to 1 content 1|0|1 
From 1 to 2 content 1|0|1 
From 1 to 3 content 1|0|1 
From 1 to 4 content 1|0|1 
From 2 to 0 content 1|0|2 
From 2 to 1 content 1|0|2 
From 2 to 2 content 1|0|2 
From 2 to 3 content 1|0|2 
From 2 to 4 content 1|0|2 
From 3 to 0 content 1|0|3 
From 3 to 1 content 1|0|3 
From 3 to 2 content 1|0

In [3]:
# Demo 2: Add adversary
setting2 = SynchronousByzantine(n=5, input=1,
                               protocol=DolevStrong,
                               measure=[ByzValidity, ByzConsistency])
exp2 = Experiment(setting2)
exp2.run_and_report()

# Adding adversary
setting2a = setting2.clone()

setting2a.f = 2 # num of adversaries
setting2a.adversary = CrashAdversary

exp2a = Experiment(setting2a)
exp2a.run_and_report()

Experiment #2
Experiment Setting:
Synchronous Byzantine
Controller : Synchronous Byzantine Controller
Environment : Synchronous Byzantine Environment
PKI : IdealPKI
Protocol : Dolev-Strong Protocol

Parameters:
n : 5
input : 1

Experiment Result:
Node 0 : 1
Node 1 : 1
Node 2 : 1
Node 3 : 1
Node 4 : 1

Measurements:
Validity : True
Consistency : True

Message History :
Round 1 : 
From 0 to 0 content 1|0 
From 0 to 1 content 1|0 
From 0 to 2 content 1|0 
From 0 to 3 content 1|0 
From 0 to 4 content 1|0 

Experiment #3
Experiment Setting:
Synchronous Byzantine
Controller : Synchronous Byzantine Controller
Environment : Synchronous Byzantine Environment
PKI : IdealPKI
Protocol : Dolev-Strong Protocol
Adversary : Crash Adversary

Parameters:
n : 5
input : 1
num of corruptions : 2
sender corrupted : False

Experiment Result:
Node 0 : 1
Node 1 : 1
Node 2 : 1

Measurements:
Validity : True
Consistency : True

Message History :
Round 1 : 
From 0 to 0 content 1|0 
From 0 to 1 content 1|0 
From 0

In [4]:
# Demo 3: Change Settings
setting3 = setting2a.clone()
setting3.f = 1
setting3.corrupt_sender = True

exp3 = Experiment(setting3)
exp3.run_and_report()

setting3.adversary = HalfHalfSender

exp3a = Experiment(setting3)
exp3a.run_and_report()

setting3.protocol = NaiveVoting

exp3b = Experiment(setting3)
exp3b.run_and_report()

Experiment #4
Experiment Setting:
Synchronous Byzantine
Controller : Synchronous Byzantine Controller
Environment : Synchronous Byzantine Environment
PKI : IdealPKI
Protocol : Dolev-Strong Protocol
Adversary : Crash Adversary

Parameters:
n : 5
input : 1
num of corruptions : 1
sender corrupted : True

Experiment Result:
Node 1 : 0
Node 2 : 0
Node 3 : 0
Node 4 : 0

Measurements:
Validity : True
Consistency : True

Message History :
Round 1 : 
Round 2 : 

Experiment #5
Experiment Setting:
Synchronous Byzantine
Controller : Synchronous Byzantine Controller
Environment : Synchronous Byzantine Environment
PKI : IdealPKI
Protocol : Dolev-Strong Protocol
Adversary : Half-half Sender

Parameters:
n : 5
input : 1
num of corruptions : 1
sender corrupted : True

Experiment Result:
Node 1 : 0
Node 2 : 0
Node 3 : 0
Node 4 : 0

Measurements:
Validity : True
Consistency : True

Message History :
Round 1 : 
From 0 to 1 content 1|0 
From 0 to 2 content 1|0 
From 0 to 3 content 0|0 
From 0 to 4 content 

In [5]:
# Demo 4: Write a protocol


class DolevStrong_Wrong:

    name = "Dolev-Strong Protocol (Wrongly Modified)"
    SENDER = 0

    def __init__(self, env, pki):
        self.env = env
        self.pki = pki
        self.pki.register(self)
        self.input = None
        self.seen = [False, False]

    def my_verify(self, round, msg):
        sigs = []
        mark = set()
        tmp = msg
        for i in range(round):
            if type(tmp) is not Message:
                return False
            s = tmp.get_sender()
            sigs.append(s)
            tmp = tmp.get_content()
            if s in mark:
                return False
            mark.add(s)
        if sigs[-1] != DolevStrong.SENDER:
            return False
        return True


    def run_node(self):
        round = self.env.get_round()
        myid = self.env.get_id(self)
        if round == 0:
            if self.env.get_id(self) == DolevStrong.SENDER:
                self.input = self.env.get_input(self)
                self.env.put_broadcast(self, self.pki.sign(self, Message(myid, self.input)))
                self.seen[self.input] = True
        elif 1 <= round <= self.env.get_f():
            msgs = self.env.get_input_msgs(self)
            for msg in msgs:
                if self.pki.verify(msg) and self.my_verify(round, msg):
                    b = msg.get_extraction()
                    if (b == 0 or b == 1) and not self.seen[b]:
                        self.seen[b] = True
                        self.env.put_broadcast(self, self.pki.sign(self, Message(myid, msg)))
            if round == self.env.get_f():
                if self.seen[1] and not self.seen[0]:
                    self.env.put_output(self, 1)
                else:
                    self.env.put_output(self, 0)
        else:
            raise RuntimeError


class RoundfAdversary:

    name = "Round f Adversary"
    adv_pool = []
    SENDER = 0
    MASTER = 0

    @staticmethod
    def clear():
        adv_pool = []

    def __init__(self, env, pki):
        self.env = env
        self.pki = pki
        self.pki.register(self)
        RoundfAdversary.adv_pool.append(self)
        self.input = None

    def run_node(self):
        round = self.env.get_round()
        myid = self.env.get_id(self)
        if round == 0:
            if self.env.get_id(self) == RoundfAdversary.SENDER:
                for i in range(1, self.env.get_n() - self.env.get_f() + 1):
                    self.env.put_packet(self, self.pki.sign(self, Message(myid, 1)), i)
        elif round == self.env.get_f() - 1:
            if self.env.get_id(self) == RoundfAdversary.MASTER:
                msg = self.pki.sign(self, Message(myid, 0))
                for node in RoundfAdversary.adv_pool:
                    if node is not self:
                        msg = self.pki.sign(node, Message(self.env.get_id(node), msg))
            if self.env.get_id(self) == RoundfAdversary.SENDER:
                for i in range(1, self.env.get_n() - self.env.get_f()):
                    self.env.put_packet(self, msg, i)
        else:
            pass

setting4 = setting3.clone()
setting4.f = 3
setting4.protocol = DolevStrong_Wrong
setting4.adversary = RoundfAdversary

exp4 = Experiment(setting4)
exp4.run_and_report()


Experiment #7
Experiment Setting:
Synchronous Byzantine
Controller : Synchronous Byzantine Controller
Environment : Synchronous Byzantine Environment
PKI : IdealPKI
Protocol : Dolev-Strong Protocol (Wrongly Modified)
Adversary : Round f Adversary

Parameters:
n : 5
input : 1
num of corruptions : 3
sender corrupted : True

Experiment Result:
Node 1 : 0
Node 2 : 1

Measurements:
Validity : True
Consistency : False

Message History :
Round 1 : 
From 0 to 1 content 1|0 
From 0 to 2 content 1|0 
Round 2 : 
From 1 to 0 content 1|0|1 
From 1 to 1 content 1|0|1 
From 1 to 2 content 1|0|1 
From 1 to 3 content 1|0|1 
From 1 to 4 content 1|0|1 
From 2 to 0 content 1|0|2 
From 2 to 1 content 1|0|2 
From 2 to 2 content 1|0|2 
From 2 to 3 content 1|0|2 
From 2 to 4 content 1|0|2 
Round 3 : 
From 4 to 1 content 0|0|3|4 

